In [1]:
from seal import *
import numpy as np
import copy

parms = EncryptionParameters (scheme_type.bgv)
poly_modulus_degree = 8192
parms.set_poly_modulus_degree(poly_modulus_degree)
parms.set_coeff_modulus(CoeffModulus.BFVDefault(poly_modulus_degree))
parms.set_plain_modulus(PlainModulus.Batching(poly_modulus_degree, 22)) #  NOTE: Standard value of 20 was not sufficient for 100x100x100 multiplication
                                                                        #  -> increased to 22
print(PlainModulus.Batching(poly_modulus_degree, 22).value())

    
context = SEALContext(parms)

keygen = KeyGenerator(context)
secret_key = keygen.secret_key()
public_key = keygen.create_public_key()
relin_keys = keygen.create_relin_keys()
galois_keys = keygen.create_galois_keys()      # Galois Keys anlegen

encryptor = Encryptor(context, public_key)
evaluator = Evaluator(context)
decryptor = Decryptor(context, secret_key)

batch_encoder = BatchEncoder(context)
slot_count = batch_encoder.slot_count()
row_size = slot_count / 2

4079617


In [2]:
class ResStorage:
    execTimeTotal = 0
    deviationTotal = 0

import time
import random

num_execs = 500
num_slots = poly_modulus_degree

ccSum = ResStorage()
ccSub = ResStorage()
ccMul = ResStorage()
cSq   = ResStorage()
cNeg  = ResStorage()
cPow  = ResStorage()
cRotR = ResStorage()
cRotL = ResStorage()
cpSum = ResStorage()
cpSub = ResStorage()
cpMul = ResStorage()

def setup():
    global v1, p1, c1, v2, p2, c2
    v1 = [0] * num_slots
    v2 = [0] * num_slots
    for i in range(num_slots):
        #v1[i] = random.randint(1,100)
        v1[i] = 100
    p1 = batch_encoder.encode(v1)
    c1 = encryptor.encrypt(p1)

    for i in range(num_slots):
        #v2[i] = random.randint(1,100)
        v2[i] = 100
    p2 = batch_encoder.encode(v2)
    c2 = encryptor.encrypt(p2)
    
# Multiplication
def fun_ccMul():
    setup()
    start = time.time()

    c3 = evaluator.multiply(c1, c2)
    evaluator.relinearize_inplace(c3, relin_keys)
    c3 = evaluator.multiply(c3, c2)
    evaluator.relinearize_inplace(c3, relin_keys)
    
    end = time.time()
    ccMul.execTimeTotal += end - start
    
    decrRes = decryptor.decrypt(c3)
    res = batch_encoder.decode(decrRes)
    for i in range(num_slots):
        actualRes = v1[i] * v2[i] * v2[i]
        ccMul.deviationTotal += abs(res[i] - actualRes)
    
# Addition
def fun_ccSum():
    setup()
    start = time.time()
    
    c3 = evaluator.add(c1, c2)
    c3 = evaluator.add(c3, c2)
    
    end = time.time()
    ccSum.execTimeTotal += end - start
    
    decrRes = decryptor.decrypt(c3)
    res = batch_encoder.decode(decrRes)
    for i in range(num_slots):
        actualRes = v1[i] + v2[i] + v2[i]
        ccSum.deviationTotal += abs(res[i] - actualRes)
    
# Subtraction
def fun_ccSub():
    setup()
    start = time.time()
    
    c3 = evaluator.sub(c1, c2)
    c3 = evaluator.sub(c3, c2)
    
    end = time.time()
    ccSub.execTimeTotal += end - start
    
    decrRes = decryptor.decrypt(c3)
    res = batch_encoder.decode(decrRes)
    for i in range(num_slots):
        actualRes = v1[i] - v2[i] - v2[i]
        ccSub.deviationTotal += abs(res[i] - actualRes)
    
# Squaring
def fun_cSq():
    setup()
    start = time.time()
    
    c3 = evaluator.square(c1)
    
    end = time.time()
    cSq.execTimeTotal += end - start
    
    decrRes = decryptor.decrypt(c3)
    res = batch_encoder.decode(decrRes)
    for i in range(num_slots):
        actualRes = pow(v1[i],2)
        cSq.deviationTotal += abs(res[i] - actualRes)
    
# Negation
def fun_cNeg():
    setup()
    start = time.time()
    
    c3 = evaluator.negate(c1)
    c3 = evaluator.negate(c3)
    
    end = time.time()
    cNeg.execTimeTotal += end - start
    
    decrRes = decryptor.decrypt(c3)
    res = batch_encoder.decode(decrRes)
    for i in range(num_slots):
        actualRes = v1[i]
        cNeg.deviationTotal += abs(res[i] - actualRes)
        
# Left Rotation

# Since Pyfhel also uses SEAL as the underlying framework, we have the same situation here:
# For some reason, the array is split in halves, and each halve gets rotated individually.
# So, if we rotate left twice, we need to iterate the two halves separately. We can still take our formulas, but have to
# adapt for the two halves.
def fun_cRotL():
    setup()
    start = time.time()
    
    c3 = evaluator.rotate_rows(c1, 1, galois_keys)
    c3 = evaluator.rotate_rows(c3, 1, galois_keys)
    
    end = time.time()
    cRotL.execTimeTotal += end - start
    
    decrRes = decryptor.decrypt(c3)
    res = batch_encoder.decode(decrRes)
        
    # First half:
    for i in range(num_slots // 2):
        actualRes = 0
        if (i + 2 >= num_slots // 2):
            actualRes = v1[i + 2 - num_slots // 2]
        else:
            actualRes = v1[i + 2]
        cRotL.deviationTotal += abs(res[i] - actualRes)
    # Second half:
    for i in range(num_slots // 2 + 1, num_slots):
        actualRes = 0
        if (i + 2 >= num_slots):
            actualRes = v1[i + 2 - num_slots // 2]
        else:
            actualRes = v1[i + 2]
        cRotL.deviationTotal += abs(res[i] - actualRes)
    
# Right Rotation

# Similar to Left Rotation, but in the second half we have to subtract num_slots / 2 to
# get a negative number how far we are out of range of the second half.
def fun_cRotR():
    setup()
    start = time.time()
    
    c3 = evaluator.rotate_rows(c1, -1, galois_keys)
    c3 = evaluator.rotate_rows(c3, -1, galois_keys)
    
    end = time.time()
    cRotR.execTimeTotal += end - start
    
    decrRes = decryptor.decrypt(c3)
    res = batch_encoder.decode(decrRes)
    
    # First half:
    for i in range(num_slots // 2):
        actualRes = 0
        if (i - 2 < 0):
            actualRes = v1[num_slots // 2 + (i - 2)]
        else:
            actualRes = v1[i - 2]
        cRotL.deviationTotal += abs(res[i] - actualRes)
    # Second half:
    for i in range(num_slots // 2 + 1, num_slots):
        actualRes = 0
        if (i - 2 < num_slots // 2):
            actualRes = v1[num_slots + (i - num_slots // 2 - 2)]
        else:
            actualRes = v1[i - 2]
        cRotL.deviationTotal += abs(res[i] - actualRes)
    
# Ciphertext-Plain Addition
def fun_cpSum():
    setup()
    start = time.time()
    
    c3 = evaluator.add_plain(c1, p2)
    c3 = evaluator.add_plain(c3, p2)
    
    end = time.time()
    cpSum.execTimeTotal += end - start
    
    decrRes = decryptor.decrypt(c3)
    res = batch_encoder.decode(decrRes)
    for i in range(num_slots):
        actualRes = v1[i] + v2[i] + v2[i]
        cpSum.deviationTotal += abs(res[i] - actualRes)
    
# Ciphertext-Plain Subtraction
def fun_cpSub():
    setup()
    start = time.time()
    
    c3 = evaluator.sub_plain(c1, p2)
    c3 = evaluator.sub_plain(c3, p2)
    
    end = time.time()
    cpSub.execTimeTotal += end - start
    
    decrRes = decryptor.decrypt(c3)
    res = batch_encoder.decode(decrRes)
    for i in range(num_slots):
        actualRes = v1[i] - v2[i] - v2[i]
        cpSub.deviationTotal += abs(res[i] - actualRes)
    
# Ciphertext-Plain Multiplication
def fun_cpMul():
    setup()
    start = time.time()
    
    c3 = evaluator.multiply_plain(c1, p2)
    evaluator.relinearize_inplace(c3, relin_keys)
    c3 = evaluator.multiply_plain(c3, p2)
    evaluator.relinearize_inplace(c3, relin_keys)
    
    end = time.time()
    cpMul.execTimeTotal += end - start
    
    decrRes = decryptor.decrypt(c3)
    res = batch_encoder.decode(decrRes)
    for i in range(num_slots):
        actualRes = v1[i] * v2[i] * v2[i]
        cpMul.deviationTotal += abs(res[i] - actualRes)
    
if __name__ == "__main__":
    
    for i in range(num_execs):
        fun_ccSum()
    for i in range(num_execs):
        fun_ccSub()
    for i in range(num_execs):
        fun_ccMul()
    for i in range(num_execs):
        fun_cSq()
    for i in range(num_execs):
        fun_cNeg()
    for i in range(num_execs):
        fun_cRotL()
    for i in range(num_execs):
        fun_cRotR()
    for i in range(num_execs):
        fun_cpSum()
    for i in range(num_execs):
        fun_cpSub()
    for i in range(num_execs):
        fun_cpMul()

In [3]:
    print('Execution time in milliseconds:')
    print()
    print('Addition:')    # Formula: Total time or deviation for all cycles / operations per cycle
    print('Average execution time: {:.6f}'.format( ( ccSum.execTimeTotal / (num_execs*2) ) * 1000 ))
    print('Average deviation:      {:.6f}'.format(   ccSum.deviationTotal / (num_execs*2) ))
    print()
    print('Subtraction:')
    print('Average execution time: {:.6f}'.format( ( ccSub.execTimeTotal / (num_execs*2) ) * 1000 ))
    print('Average deviation:      {:.6f}'.format(   ccSub.deviationTotal / (num_execs*2) ))
    print()
    print('Multiplication:')
    print('Average execution time: {:.6f}'.format( ( ccMul.execTimeTotal / (num_execs*2) ) * 1000 ))
    print('Average deviation:      {:.6f}'.format(   ccMul.deviationTotal / (num_execs*2) ))
    print()
    print('Squaring:')    # Square only has 1 operation per cycle, otherwise the configuration needs to be changed
    print('Average execution time: {:.6f}'.format( ( cSq.execTimeTotal / (num_execs) ) * 1000 ))
    print('Average deviation:      {:.6f}'.format(   cSq.deviationTotal / (num_execs) ))
    print()
    print('Negation:')
    print('Average execution time: {:.6f}'.format( ( cNeg.execTimeTotal / (num_execs*2) ) * 1000 ))
    print('Average deviation:      {:.6f}'.format(   cNeg.deviationTotal / (num_execs*2) ))
    print()
    print('Left Rotation:')
    print('Average execution time: {:.6f}'.format( ( cRotL.execTimeTotal / (num_execs*2) ) * 1000 ))
    print('Average deviation:      {:.6f}'.format(   cRotL.deviationTotal / (num_execs*2) ))
    print()
    print('Right Rotation:')
    print('Average execution time: {:.6f}'.format( ( cRotR.execTimeTotal / (num_execs*2) ) * 1000 ))
    print('Average deviation:      {:.6f}'.format(   cRotR.deviationTotal / (num_execs*2) ))
    print()
    print('Ciphertext-Plain Addition:')
    print('Average execution time: {:.6f}'.format( ( cpSum.execTimeTotal / (num_execs*2) ) * 1000 ))
    print('Average deviation:      {:.6f}'.format(   cpSum.deviationTotal / (num_execs*2) ))
    print()
    print('Ciphertext-Plain Subtraction:')
    print('Average execution time: {:.6f}'.format( ( cpSub.execTimeTotal / (num_execs*2) ) * 1000 ))
    print('Average deviation:      {:.6f}'.format(   cpSub.deviationTotal / (num_execs*2) ))
    print()
    print('Ciphertext-Plain Multiplication:')
    print('Average execution time: {:.6f}'.format( ( cpMul.execTimeTotal / (num_execs*2) ) * 1000 ))
    print('Average deviation:      {:.6f}'.format(   cpMul.deviationTotal / (num_execs*2) ))
    
    PRINT_RAW_NUMBERS = True
    if (PRINT_RAW_NUMBERS):
        print()
        print()
        print('Raw numbers:')
        print()
        print('Average execution time:')
        print('{:.6f}'.format( ( ccSum.execTimeTotal / (num_execs*2) ) * 1000 ))
        print('{:.6f}'.format( ( ccSub.execTimeTotal / (num_execs*2) ) * 1000 ))
        print('{:.6f}'.format( ( ccMul.execTimeTotal / (num_execs*2) ) * 1000 ))
        print('{:.6f}'.format( ( cSq.execTimeTotal / (num_execs) ) * 1000 ))
        print('{:.6f}'.format( ( cNeg.execTimeTotal / (num_execs*2) ) * 1000 ))
        print('{:.6f}'.format( ( cRotL.execTimeTotal / (num_execs*2) ) * 1000 ))
        print('{:.6f}'.format( ( cRotR.execTimeTotal / (num_execs*2) ) * 1000 ))
        print('{:.6f}'.format( ( cpSum.execTimeTotal / (num_execs*2) ) * 1000 ))
        print('{:.6f}'.format( ( cpSub.execTimeTotal / (num_execs*2) ) * 1000 ))
        print('{:.6f}'.format( ( cpMul.execTimeTotal / (num_execs*2) ) * 1000 ))
        print()
        print('Average deviation:')
        print('{:.6f}'.format(   ccSum.deviationTotal / (num_execs*2) ))
        print('{:.6f}'.format(   ccSub.deviationTotal / (num_execs*2) ))
        print('{:.6f}'.format(   ccMul.deviationTotal / (num_execs*2) ))
        print('{:.6f}'.format(   cSq.deviationTotal / (num_execs) ))
        print('{:.6f}'.format(   cNeg.deviationTotal / (num_execs*2) ))
        print('{:.6f}'.format(   cRotL.deviationTotal / (num_execs*2) ))
        print('{:.6f}'.format(   cRotR.deviationTotal / (num_execs*2) ))
        print('{:.6f}'.format(   cpSum.deviationTotal / (num_execs*2) ))
        print('{:.6f}'.format(   cpSub.deviationTotal / (num_execs*2) ))
        print('{:.6f}'.format(   cpMul.deviationTotal / (num_execs*2) ))

Execution time in milliseconds:

Addition:
Average execution time: 0.098249
Average deviation:      0.000000

Subtraction:
Average execution time: 0.106374
Average deviation:      0.000000

Multiplication:
Average execution time: 8.752813
Average deviation:      0.000000

Squaring:
Average execution time: 3.713743
Average deviation:      0.000000

Negation:
Average execution time: 0.136612
Average deviation:      0.000000

Left Rotation:
Average execution time: 5.917861
Average deviation:      0.000000

Right Rotation:
Average execution time: 6.636212
Average deviation:      0.000000

Ciphertext-Plain Addition:
Average execution time: 0.141598
Average deviation:      0.000000

Ciphertext-Plain Subtraction:
Average execution time: 0.170619
Average deviation:      0.000000

Ciphertext-Plain Multiplication:
Average execution time: 0.238753
Average deviation:      0.000000


Raw numbers:

Average execution time:
0.098249
0.106374
8.752813
3.713743
0.136612
5.917861
6.636212
0.141598
0.1706